In [9]:
import xml.etree.ElementTree as et
import regex as re
from tqdm import tqdm
import json
import xmltodict
import os


In [10]:
# Extract CPC codes from patent documents (just data exploration)
def check_cpc(xml_string):
    # for i, xml_string in tqdm(enumerate(xml_strings)):
        doc = et.fromstring(xml_string)
        cpc_tags = []
        Y02_tags=[]
        # Iterate over all CPC XML tags in the current document
        for elem in doc.iter("classification-cpc-text"):
            cpc_tags.append(elem.text)
        # cpc_map[i] = cpc_tags
        for tag in cpc_tags:
            if tag.startswith("Y02"):
                Y02_tags.append(tag)
        return Y02_tags

In [20]:
# The weekly data dumps from USPTO contain a lot of patent xml documents concatenated in one big xml file
# --> We need to extract the individual documents
# Store each patent xml document as a string in xml_strings
xml_strings = []
  # Define SOS sequence that indicates start of new patent
XML_SOS = re.escape("<?xml") + "*"
cpc_map = {}
input= "/Users/ishitachakravarthy/Desktop/CS685-NLP/project/patent-topic-gpt/data/raw/ipg240326.xml"

count=0
with open(input) as file:
    count=count+1
    xml_string = ""
    i = 0
    cpc_tags=[]
    for line in file:
      if (re.match(XML_SOS, line) != None) & (xml_string != ""):
          i += 1
          cpc_tags=check_cpc(xml_string)
          if cpc_tags!=[]:
            xml_strings.append(xml_string)
          xml_string = ""
          count=count+1
      xml_string += line

        

In [19]:

xml=[]
data_dict={}
for i,v in enumerate(xml_strings):
    print(i)
    v=v.split('\n')
    for ind,line in enumerate(v):
        xml_string = re.sub("<(b|u|i|(sup)|(sub))>", "", line)
        xml_string = re.sub("<\/(b|u|i|(sup)|(sub))>", "", xml_string)
        xml_string = re.sub(" num=\"[0-9]*\"", "", xml_string)
        xml_string = re.sub(" id=\"((p-)|(ul)|(ul[0-9]*-))[0-9]*\"", "", xml_string)
        xml_string = re.sub(" id=\".*\"", "", xml_string)
        xml_string = re.sub("<li>", "", xml_string)
        xml_string = re.sub("<\/li>", "", xml_string)
        xml_string = re.sub("<ul list-style=\"none\">", "", xml_string)
        xml_string = re.sub("<\/ul>", "", xml_string)
        xml_string = re.sub("<p>", "", xml_string)
        xml_string = re.sub("<\/p>", "", xml_string)
        v[ind]=xml_string

    str=""
    for line in v:
        str=str+line+ '\n'
    xml_strings[i]=str

    str=""
    for line in v:
        str=str+line


print(len(xml_strings))

0


In [33]:

count=0
data_arr=[]
for ind,val in enumerate(xml_strings):
    try:
      data_dict[ind]=xmltodict.parse(val)
      data_arr.append(xmltodict.parse(val)['us-patent-grant'])
    except:
       count=count+1
       print(f"skipping index {ind}")
print(count)

skipping index 15
skipping index 39
2


In [35]:
json_str=json.dumps(data_dict)

with open("sample.json", "a") as outfile:
        outfile.write(json_str)

JSONDecodeError: Extra data: line 1 column 12629855 (char 12629854)

In [36]:
patent_number=""
abstract=""
cpc_obj=""
cpc_id=[]
final_json=[]
print(len(data_arr))
for i,v in enumerate(data_arr):
    try:
        patent_number=v["us-bibliographic-data-grant"]["publication-reference"]["document-id"]["country"]+'-'+v["us-bibliographic-data-grant"]["publication-reference"]["document-id"]["doc-number"]+'-'+v["us-bibliographic-data-grant"]["publication-reference"]["document-id"]["kind"]
        abstract=v["abstract"]
        cpc_id=v["us-bibliographic-data-grant"]["us-field-of-classification-search"]['classification-cpc-text']
        obj={}
        obj["patent_number"]=patent_number
        obj["abstract"]=abstract
        obj["cpc_id"]=cpc_id
        final_json.append(obj)
    except:
        print(f"skipping{i}")
print(len(final_json))

149
skipping0
skipping1
skipping2
skipping3
skipping82
skipping84
143


In [38]:
with open("xyz.json", "a") as outfile:
    for entry in final_json:
        print(entry)
        json_str=json.dump(entry,outfile)
        outfile.write('\n')



{'patent_number': 'US-11937550-B2', 'abstract': 'A drum assembly for turning material. The drum assembly includes a drum having a first end, a second end, a rotational axis extending through the first end and the second end about which the drum rotates. The drum includes a first outer portion, a second outer portion, and a central portion. A plurality of centering paddles are connected to the drum and configured to move material toward a central plane. A plurality of dispersal paddles are connected to the drum and configured to move material away from the drum in a direction generally perpendicular to the rotational axis. A plurality of spreading paddles are connected to the drum and configured to move material away from the central plane.', 'cpc_id': ['B02C 21/00', 'B02C 21/002', 'B02C 2021/023', 'A01B 9/00-006', 'A01B 33/00-025', 'A01B 39/14', 'A01D 78/006', 'A01D 78/00', 'Y02W 30/40', 'C05F 17/943', 'Y02P 20/145', 'B01F 27/70', 'B01F 33/5021']}
{'patent_number': 'US-11937560-B2', 'a

In [480]:
with open('xyz.json') as json_file:
    data = json.load(json_file)
    
print(len(data))


JSONDecodeError: Expecting value: line 1 column 1 (char 0)